### [MAPLEAF.IO](https://henrystoldt.github.io/MAPLEAF/IO/index.html)
is the module in charge of simulation input/output.
Let's have a look at the [SimDefinition](https://henrystoldt.github.io/MAPLEAF/IO/simDefinition.html#MAPLEAF.IO.simDefinition.SimDefinition) & [SubDictReader](https://henrystoldt.github.io/MAPLEAF/IO/subDictReader.html#MAPLEAF.IO.subDictReader.SubDictReader) classes, which are responsible for the input side of things.

MAPLEAF uses a simple text format to define simulations (.mapleaf files).  
The SimDefinition class reads these files and makes their contents accessible to the rest of MAPLEAF.

#### Reading/Editing/Writing .mapleaf files
Let's make an example file:

In [1]:
with open("foo.mapleaf", "w+") as foo:
    foo.write("""
    Rocket{
        OnlyStage{
            class   Stage
            
            Nosecone{
                class       Nosecone
                aspectRatio 4
                donuts      Are Great
            }
        }
    }""")    

Let's parse the file using SimDefinition:

In [2]:
from MAPLEAF.IO import SimDefinition
simDef = SimDefinition("foo.mapleaf")

We can see what it's read from the file by printing it:

In [3]:
print(simDef)

File: foo.mapleaf
Rocket.OnlyStage.class: Stage
Rocket.OnlyStage.Nosecone.class: Nosecone
Rocket.OnlyStage.Nosecone.aspectRatio: 4
Rocket.OnlyStage.Nosecone.donuts: Are Great




As the printed output suggests, SimDefinition does not store dictionaries themselves, only key-value pairs.
The key to each value is "Dict1.Dict2...Dictn.key" and values are STRINGS.
Keys end at the first whitespace in their lines. Everything after the first whitespace becomes the STRING value.

To obtain values, we just need to know the path to the key:

In [4]:
simDef.getValue("Rocket.OnlyStage.Nosecone.aspectRatio")

'4'

SimDefinition doesn't check whether values are correct or required for the simulation, it parses everything:

In [5]:
simDef.getValue("Rocket.OnlyStage.Nosecone.donuts")

'Are Great'

We can also edit or add values:

In [6]:
simDef.setValue("Rocket.OnlyStage.Nosecone.aspectRatio", "4.5")
simDef.setValue("MyNewFeatureDict.key", "value")

print(simDef)

File: foo.mapleaf
Rocket.OnlyStage.class: Stage
Rocket.OnlyStage.Nosecone.class: Nosecone
Rocket.OnlyStage.Nosecone.aspectRatio: 4.5
Rocket.OnlyStage.Nosecone.donuts: Are Great
MyNewFeatureDict.key: value




This is very helpful for running parametric studies -> Load file -> edit some values -> run simulation based on edited SimDefinition -> record results -> repeat.

Currently, the changes we've made have not been written to disk:

In [7]:
origSimDef = SimDefinition("foo.mapleaf")
print(origSimDef)

File: foo.mapleaf
Rocket.OnlyStage.class: Stage
Rocket.OnlyStage.Nosecone.class: Nosecone
Rocket.OnlyStage.Nosecone.aspectRatio: 4
Rocket.OnlyStage.Nosecone.donuts: Are Great




To write a new version to file:

In [8]:
simDef.writeToFile("foo_edited.mapleaf")
with open("foo_edited.mapleaf") as newFile:
    print(newFile.read())

# MAPLEAF
# File: foo_edited.mapleaf
# Autowritten on: 2020-09-20 14:57:22.423710

MyNewFeatureDict{
	key	value
}

Rocket{

	OnlyStage{

		Nosecone{
			aspectRatio	4.5
			class	Nosecone
			donuts	Are Great
		}

		class	Stage
	}
}



You'll notice that the order of some keys has changed, but the contents are the same, and the new file is ready to be parsed by another instance of SimDefinition.  
**WARNING**: Comments are deleted by SimDefinition reading->writing!

So that's the basics of reading/writing data.  
A few important features make SimDefinitions more convenient to use in actual simulations:
1. Defaults
2. Introspection
3. SubDictReader

#### Default Values:
Default values can make SimDefinitions act like they contain values that weren't actually defined in the file they parsed:

In [9]:
print(simDef.getValue("Environment.MeanWindModel"))
print(simDef.getValue("Environment.ConstantMeanWind.velocity"))

Constant
(0 0 0)


SimDefinition grabs these values from one giant dictionary: MAPLEAF.IO.simDefinition.defaultConfigValues:

In [10]:
from MAPLEAF.IO import defaultConfigValues
print(defaultConfigValues["Environment.MeanWindModel"])
print(defaultConfigValues["Environment.ConstantMeanWind.velocity"])

Constant
(0 0 0)


Note that there are two types of default keys:
1. Absolute (Start at the root of a .mapleaf file)
2. Class-based (Relative to a subdictionary with a 'class')

Example of a class-based default:

In [11]:
print(simDef.getValue("Rocket.OnlyStage.Nosecone.class")) # The Nosecone dictionary has a 'class' key defined
print(simDef.getValue("Rocket.OnlyStage.Nosecone.shape")) # When we ask for .shape, it retrieves Nosecone.shape from the default dict
print(defaultConfigValues["Nosecone.shape"])

Nosecone
tangentOgive
tangentOgive


#### Instrospection
Some dictionaries don't have a pre-defined structure (ex. Rocket can have any number of stages, each stage can have any number of components), so we don't know the paths to all the keys we need to read in advance.

getImmediateSubKeys and getImmediateSubDicts help MAPLEAF discover the contents of a SimDefinition:

`getImmediateSubKeys(path)` finds child keys:

In [12]:
print(simDef.getImmediateSubKeys("Rocket"))
print(simDef.getImmediateSubKeys("Rocket.OnlyStage"))
print(simDef.getImmediateSubKeys("Rocket.OnlyStage.Nosecone"))

[]
['Rocket.OnlyStage.class']
['Rocket.OnlyStage.Nosecone.donuts', 'Rocket.OnlyStage.Nosecone.class', 'Rocket.OnlyStage.Nosecone.aspectRatio']


`getImmediateSubDicts(path)` finds child dictionaries:

In [13]:
print("Top-Level dicts: " + str(simDef.getImmediateSubDicts("")))
print("Stages: " +          str(simDef.getImmediateSubDicts("Rocket")))
print("Components: " +      str(simDef.getImmediateSubDicts("Rocket.OnlyStage")))

Top-Level dicts: ['MyNewFeatureDict', 'Rocket']
Stages: ['Rocket.OnlyStage']
Components: ['Rocket.OnlyStage.Nosecone']


#### SubDictReader
SubDictReader is a wrapper around SimDefinition that makes reading and parsing values from a single dictionary more convenient.  
Most times when MAPLEAF reads values from a SimDefinition, it is through a SubDictReader.
Point it at a subdictionary when creating one:

In [14]:
from MAPLEAF.IO import SubDictReader
noseConeDictReader = SubDictReader("Rocket.OnlyStage.Nosecone", simDef)

Then you can easily read & parse values from that dictionary:

In [15]:
noseConeDictReader.getFloat("aspectRatio")

4.5

#### Monte Carlo
SimDefinition's final important purpose in MAPLEAF is to make some parameters probabilistic, for Monte Carlo simulations. This happens automatically for scalar and vector parameters, when a _stdDev key is present:

In [16]:
# Scalar
simDef.setValue("Rocket.OnlyStage.Nosecone.mass", "5") # This becomes avg of normal distribution
simDef.setValue("Rocket.OnlyStage.Nosecone.mass_stdDev", "0.2") # Standard deviation

noseConeDictReader.getFloat("mass")
noseConeDictReader.getFloat("mass")
noseConeDictReader.getFloat("mass")

print("")

# Vector
simDef.setValue("Rocket.velocity", "(0 0 10)") # This becomes avg of normal distribution
simDef.setValue("Rocket.velocity_stdDev", "(0.1 0.1 2)") # Standard deviation

noseConeDictReader.getVector("Rocket.velocity")
noseConeDictReader.getVector("Rocket.velocity")
noseConeDictReader.getVector("Rocket.velocity")

Sampling scalar parameter: Rocket.OnlyStage.Nosecone.mass, value: 4.490
Sampling scalar parameter: Rocket.OnlyStage.Nosecone.mass, value: 5.267
Sampling scalar parameter: Rocket.OnlyStage.Nosecone.mass, value: 5.130

Sampling vector parameter: Rocket.velocity, value: (0.098 0.052 10.083)
Sampling vector parameter: Rocket.velocity, value: (0.162 0.109 6.917)
Sampling vector parameter: Rocket.velocity, value: (-0.129 0.105 8.485)


If we remove the _stdDev keys, the values are deterministic again:

In [17]:
simDef.removeKey("Rocket.OnlyStage.Nosecone.mass_stdDev")
simDef.removeKey("Rocket.velocity_stdDev")

print(noseConeDictReader.getFloat("mass"))
print(noseConeDictReader.getFloat("mass"))
print(noseConeDictReader.getFloat("mass"))

print(noseConeDictReader.getVector("Rocket.velocity"))
print(noseConeDictReader.getVector("Rocket.velocity"))
print(noseConeDictReader.getVector("Rocket.velocity"))

5.0
5.0
5.0
(0.0 0.0 10.0)
(0.0 0.0 10.0)
(0.0 0.0 10.0)


#### Access/Use Tracking
Finally, SimDefinition keeps track of what fields are accessed and what default values were used during a simulation. You'll see output like the following after running a simulation:

In [18]:
simDef.printUnusedKeys()
simDef.printDefaultValuesUsed()


Rocket.OnlyStage.class:                      Stage


Environment.ConstantMeanWind.velocity:       (0 0 0)
Environment.MeanWindModel:                   Constant
Nosecone.shape:                              tangentOgive

If this was not intended, override the default values by adding the above information to your simulation definition file.

